In [1]:
%matplotlib inline

import matplotlib.pyplot as plt

import sys
sys.path.append("/home/ubuntu/Notebooks/annsa/")
import os
import pickle

import tensorflow.contrib.eager as tfe
from sklearn import datasets, preprocessing, model_selection
from sklearn.model_selection import KFold

import annsa as an
import tensorflow as tf
import numpy as np

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

#### Define neural network

In [3]:
from cvt_oct_tensorflow_classes import fc_nn,fc_nn_model_features,class_isotopes,train_kfolds

#### Get some data

In [4]:
scaler=preprocessing.StandardScaler()
mlb=preprocessing.MultiLabelBinarizer()

In [5]:
training_data = np.load('CVT-oct-spectra-test.npy')
training_keys = np.load('CVT-oct-keys-test.npy')
nan_isotopes=list(set(np.argwhere(np.isnan(training_data))[:,0]))
training_data=np.delete(training_data,nan_isotopes,axis=0)
training_keys=np.delete(training_keys,nan_isotopes,axis=0)

training_data = scaler.fit_transform(training_data)
training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_data.shape[0],1]))

#### Define an optimizer, model, and train the network

In [13]:
number_hyperparameters_to_search = 5
final_avg_test_errors=np.zeros(number_hyperparameters_to_search)
model_id='fc_nn'

for hyperparameter_index in range(number_hyperparameters_to_search):
    print("Searching hyperparameter combination number "+str(hyperparameter_index))
    
    model_features = fc_nn_model_features(learining_rate=4.5e-3,
                                        l2_regularization_scale=1.09e-3,
                                        dropout_probability=1.0-0.96,
                                        nodes_layer_1=449,
                                        nodes_layer_2=305)

    with open('./hyperparameter-search-results/'+model_id+'_'+str(hyperparameter_index)+'_features', 'w+') as f:
        pickle.dump(model_features,f)
    
    _,final_avg_test_errors[hyperparameter_index]=train_kfolds(training_data=training_data,
                                                 training_keys=training_keys_binarized,
                                                 number_folds=5,
                                                 num_epochs=1,
                                                 model_class=fc_nn,
                                                 model_features=model_features,
                                                 verbose=True)
    np.save('./hyperparameter-search-results/final_avg_test_errors_'+model_id+'_'+str(hyperparameter_index),final_avg_test_errors)

Searching hyperparameter combination number 0
training loss: 3.28  testing loss: 3.28
training loss: 3.51  testing loss: 3.51
training loss: 3.39  testing loss: 3.39
training loss: 3.38  testing loss: 3.38
training loss: 3.48  testing loss: 3.48
final average training loss: 3.41 final average testing loss: 3.41
Searching hyperparameter combination number 1
training loss: 3.49  testing loss: 3.49
training loss: 3.48  testing loss: 3.48
training loss: 3.37  testing loss: 3.37
training loss: 3.38  testing loss: 3.38
training loss: 3.15  testing loss: 3.15
final average training loss: 3.37 final average testing loss: 3.37
Searching hyperparameter combination number 2
training loss: 3.49  testing loss: 3.49
training loss: 3.49  testing loss: 3.49
training loss: 3.34  testing loss: 3.34
training loss: 3.33  testing loss: 3.33
training loss: 3.36  testing loss: 3.36
final average training loss: 3.40 final average testing loss: 3.40
Searching hyperparameter combination number 3
training loss: 